In [3]:
import pandas as pd
import numpy as np

на примере датасета
поделили выборку на тренировочную и тестовую, отделили метки классов

In [23]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
df = load_iris()
X_train, X_test, y_train, y_test = train_test_split(df.data, df.target, test_size=0.3)

In [11]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [12]:
print(X_train.shape)
print(X_test.shape)

(105, 4)
(45, 4)


In [115]:
def knn_er(X_train, X_test, y_train, y_test, k):
    
    # считаем расстояние каждый элемент тестовой выборки с каждым наблюдением на тренировочной выборке
    import math as math
    each_to_each_dist = []
    for i in range(len(X_test)):
        for j in range(len(X_train)):
            each_to_each_dist.append([i, j, math.sqrt(sum((X_train[j] - X_test[i])**2))])
    
    # переход к DataFrame
    df = pd.DataFrame(each_to_each_dist)
    df = df.rename(columns={0: 'i_test', 1: 'j_train', 2:'dist'})
    
    # ранжируем расстояние в разрезе каждого тестового наблюдения
    df['rank'] = df.groupby('i_test')['dist'].rank(ascending=True)
    df = df.sort_values(by='rank', ascending = True)
    
    # формируем df с k ближайшими наблюдениями из тренировочного сета
    k_near_Neighbors = df.groupby('i_test').head(k)
    
    # добавляем столбец с метками класса по id тренировочного наблюдения
    train_target = []
    for i in k_near_Neighbors['j_train']:
        train_target.append(y_train[i]) 

    k_near_Neighbors['train_tar'] = train_target
    
    # присваиваем тестовому экземпляру класс тренировочных наблюдений, которые чаще оказываются близко к нему
    tar_count = k_near_Neighbors.groupby(['i_test', 'train_tar']).size().reset_index(name='counts')
    tar_count = tar_count.sort_values(by='counts', ascending = False)
    res = tar_count.groupby('i_test').head(1).sort_values('i_test')
    
    
    # добавляем столбец с исходными метками по тестовым наблюдениям
    res['fin_tar'] = y_test
    
    # рассчитываем качество классификатора
    res['score'] = np.where(res['train_tar']==res['fin_tar'], 1, 0)
    return (float(res['score'].sum())/res.shape[0])*100.0

In [117]:
%%time
knn_er(X_train, X_test, y_train, y_test, k = 3)

Wall time: 116 ms


C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


97.77777777777777